In [1]:
import tqdm
import pandas as pd

def make_result_df(result_file):
    with open(result_file, "r") as f:
        lines = f.readlines()
    results = {'F': {}, 'D': {}}

    id = None
    for line in tqdm.tqdm(lines):
        if line.startswith("id"):
            id = line.split()[1][1:-1].upper()
            ref = None
            hyp = None
        elif line.startswith("REF:"):
            ref = line.split()[1:]
            continue
        elif line.startswith("HYP:"):
            hyp = line.split()[1:]
            continue
        if id is not None and ref is not None and hyp is not None:
            for key in ('F', 'D'):
                tp, fn, fp = 0, 0, 0
                for r, h in zip(ref, hyp):
                    r = r.upper()
                    h = h.upper()
                    if key in r:
                        if key in h:
                            tp += 1
                        else:
                            fn += 1
                    else:
                        if key in h:
                            fp += 1
                results[key][id] = {'TP': tp, 'FN': fn, 'FP': fp}
                # if key == 'D' and fn > 0:
                #     print(id)
                #     print("REF:", ' '.join([t.upper() for t in ref]))
                #     print("HYP:", ' '.join([t.upper() for t in hyp]))
            id = None
            ref = None
            hyp = None

    result_dfs = {}
    for key, value in results.items():
        df = pd.DataFrame(value).T

        # dfの各列の合計値を，ALLというインデクスで一番最後の行に追加する
        df.loc['ALL'] = df.sum()
        # Precision, Recall, F1を計算する．ただしTP, FN, FPが0の場合は0とする．
        df['Precision'] = (df['TP'] / (df['TP'] + df['FP'])).fillna(0)
        df['Recall'] = (df['TP'] / (df['TP'] + df['FN'])).fillna(0)
        df['F1'] = (2 * df['Precision'] * df['Recall'] / (df['Precision'] + df['Recall'])).fillna(0)
        result_dfs[key] = df

    return result_dfs

In [61]:
import os

exp_name = "asr_train_asr_conformer_raw_jp_word/decode_asr_lm_lm_train_lm_jp_word_valid.loss.ave_asr_model_valid.acc.ave"
out_name = "conformer_transformer_ctc"

for eval_name in ["eval1", "eval2", "eval3"]:
    eval_dir = os.path.join("exp", exp_name, eval_name, "score_wer")
    result_file = os.path.join(eval_dir, "result.txt")
    result_dfs = make_result_df(result_file)

    for key, df in result_dfs.items():
        print(f"Eval: {eval_name}, Type: {key}")
        print(df.loc['ALL'].to_dict())
        out_csv_name = f"{out_name}_{eval_name}_{key}.csv"
        df.to_csv(out_csv_name)
        print(f"Saved to {out_csv_name}")

100%|██████████| 8501/8501 [00:00<00:00, 201384.77it/s]


Eval: eval1, Type: F
{'TP': 2980.0, 'FN': 246.0, 'FP': 151.0, 'Precision': 0.9517725966145002, 'Recall': 0.9237445753254805, 'F1': 0.9375491584080541}
Saved to conformer_transformer_ctc_eval1_F.csv
Eval: eval1, Type: D
{'TP': 291.0, 'FN': 293.0, 'FP': 50.0, 'Precision': 0.8533724340175953, 'Recall': 0.4982876712328767, 'F1': 0.6291891891891892}
Saved to conformer_transformer_ctc_eval1_D.csv


100%|██████████| 8579/8579 [00:00<00:00, 233612.94it/s]


Eval: eval2, Type: F
{'TP': 2316.0, 'FN': 166.0, 'FP': 119.0, 'Precision': 0.951129363449692, 'Recall': 0.9331184528605962, 'F1': 0.9420378279438683}
Saved to conformer_transformer_ctc_eval2_F.csv
Eval: eval2, Type: D
{'TP': 321.0, 'FN': 232.0, 'FP': 71.0, 'Precision': 0.8188775510204082, 'Recall': 0.5804701627486437, 'F1': 0.6793650793650793}
Saved to conformer_transformer_ctc_eval2_D.csv


100%|██████████| 8723/8723 [00:00<00:00, 299076.40it/s]


Eval: eval3, Type: F
{'TP': 1555.0, 'FN': 145.0, 'FP': 161.0, 'Precision': 0.9061771561771562, 'Recall': 0.9147058823529411, 'F1': 0.9104215456674474}
Saved to conformer_transformer_ctc_eval3_F.csv
Eval: eval3, Type: D
{'TP': 130.0, 'FN': 137.0, 'FP': 26.0, 'Precision': 0.8333333333333334, 'Recall': 0.4868913857677903, 'F1': 0.6146572104018914}
Saved to conformer_transformer_ctc_eval3_D.csv


In [63]:
import os

exp_name = "asr_train_asr_streaming_conformer_raw_jp_word/decode_asr_streaming_lm_lm_train_lm_jp_word_valid.loss.ave_asr_model_valid.acc.ave"
out_name = "cbs_transformer_ctc"

for eval_name in ["eval1", "eval2", "eval3"]:
    eval_dir = os.path.join("exp", exp_name, eval_name, "score_wer")
    result_file = os.path.join(eval_dir, "result.txt")
    result_dfs = make_result_df(result_file)

    for key, df in result_dfs.items():
        print(f"Eval: {eval_name}, Type: {key}")
        print(df.loc['ALL'].to_dict())
        out_csv_name = f"{out_name}_{eval_name}_{key}.csv"
        df.to_csv(out_csv_name)
        print(f"Saved to {out_csv_name}")

  0%|          | 0/8501 [00:00<?, ?it/s]

100%|██████████| 8501/8501 [00:00<00:00, 211258.45it/s]


Eval: eval1, Type: F
{'TP': 3025.0, 'FN': 201.0, 'FP': 215.0, 'Precision': 0.933641975308642, 'Recall': 0.9376937383756975, 'F1': 0.9356634704608724}
Saved to cbs_transformer_ctc_eval1_F.csv
Eval: eval1, Type: D
{'TP': 293.0, 'FN': 291.0, 'FP': 77.0, 'Precision': 0.7918918918918919, 'Recall': 0.5017123287671232, 'F1': 0.6142557651991614}
Saved to cbs_transformer_ctc_eval1_D.csv


100%|██████████| 8576/8576 [00:00<00:00, 236393.55it/s]


Eval: eval2, Type: F
{'TP': 2318.0, 'FN': 164.0, 'FP': 149.0, 'Precision': 0.9396027563842724, 'Recall': 0.9339242546333602, 'F1': 0.9367548999797939}
Saved to cbs_transformer_ctc_eval2_F.csv
Eval: eval2, Type: D
{'TP': 325.0, 'FN': 228.0, 'FP': 95.0, 'Precision': 0.7738095238095238, 'Recall': 0.5877034358047016, 'F1': 0.6680369989722508}
Saved to cbs_transformer_ctc_eval2_D.csv


100%|██████████| 8723/8723 [00:00<00:00, 348662.64it/s]


Eval: eval3, Type: F
{'TP': 1581.0, 'FN': 119.0, 'FP': 184.0, 'Precision': 0.8957507082152975, 'Recall': 0.93, 'F1': 0.9125541125541126}
Saved to cbs_transformer_ctc_eval3_F.csv
Eval: eval3, Type: D
{'TP': 121.0, 'FN': 146.0, 'FP': 32.0, 'Precision': 0.7908496732026143, 'Recall': 0.45318352059925093, 'F1': 0.5761904761904763}
Saved to cbs_transformer_ctc_eval3_D.csv


In [2]:
import os

exp_name = "asr_train_asr_cbs_transducer_090306/cbs_with_lm"
out_name = "cbs090306_transducer_with_lm"

for eval_name in ["eval1", "eval2", "eval3"]:
    eval_dir = os.path.join("exp", exp_name, eval_name, "score_wer")
    result_file = os.path.join(eval_dir, "result.txt")
    result_dfs = make_result_df(result_file)

    for key, df in result_dfs.items():
        print(f"Eval: {eval_name}, Type: {key}")
        print(df.loc['ALL'].to_dict())
        out_csv_name = f"{out_name}_{eval_name}_{key}.csv"
        df.to_csv(out_csv_name)
        print(f"Saved to {out_csv_name}")

100%|██████████| 8501/8501 [00:00<00:00, 246058.04it/s]


Eval: eval1, Type: F
{'TP': 2944.0, 'FN': 282.0, 'FP': 138.0, 'Precision': 0.9552238805970149, 'Recall': 0.9125852448853069, 'F1': 0.933417882054534}
Saved to cbs090306_transducer_with_lm_eval1_F.csv
Eval: eval1, Type: D
{'TP': 166.0, 'FN': 418.0, 'FP': 30.0, 'Precision': 0.8469387755102041, 'Recall': 0.2842465753424658, 'F1': 0.4256410256410257}
Saved to cbs090306_transducer_with_lm_eval1_D.csv


100%|██████████| 8579/8579 [00:00<00:00, 234736.12it/s]


Eval: eval2, Type: F
{'TP': 2277.0, 'FN': 205.0, 'FP': 148.0, 'Precision': 0.9389690721649484, 'Recall': 0.9174053182917002, 'F1': 0.9280619523130222}
Saved to cbs090306_transducer_with_lm_eval2_F.csv
Eval: eval2, Type: D
{'TP': 183.0, 'FN': 370.0, 'FP': 41.0, 'Precision': 0.8169642857142857, 'Recall': 0.3309222423146474, 'F1': 0.47104247104247104}
Saved to cbs090306_transducer_with_lm_eval2_D.csv


100%|██████████| 8723/8723 [00:00<00:00, 248663.90it/s]


Eval: eval3, Type: F
{'TP': 1567.0, 'FN': 133.0, 'FP': 190.0, 'Precision': 0.8918611269208879, 'Recall': 0.9217647058823529, 'F1': 0.9065663870407868}
Saved to cbs090306_transducer_with_lm_eval3_F.csv
Eval: eval3, Type: D
{'TP': 51.0, 'FN': 216.0, 'FP': 13.0, 'Precision': 0.796875, 'Recall': 0.19101123595505617, 'F1': 0.3081570996978852}
Saved to cbs090306_transducer_with_lm_eval3_D.csv


In [3]:
import os

exp_name = "asr_train_asr_cbs_transducer_081616_hop128/cbs_20epoch_with_lm"
out_name = "cbs081616hop128_transducer_with_lm"

for eval_name in ["eval1", "eval2", "eval3"]:
    eval_dir = os.path.join("exp", exp_name, eval_name, "score_wer")
    result_file = os.path.join(eval_dir, "result.txt")
    result_dfs = make_result_df(result_file)

    for key, df in result_dfs.items():
        print(f"Eval: {eval_name}, Type: {key}")
        print(df.loc['ALL'].to_dict())
        out_csv_name = f"{out_name}_{eval_name}_{key}.csv"
        df.to_csv(out_csv_name)
        print(f"Saved to {out_csv_name}")

100%|██████████| 8501/8501 [00:00<00:00, 245526.01it/s]


Eval: eval1, Type: F
{'TP': 2929.0, 'FN': 297.0, 'FP': 166.0, 'Precision': 0.9463651050080776, 'Recall': 0.9079355238685679, 'F1': 0.9267520961873121}
Saved to cbs081616hop128_transducer_with_lm_eval1_F.csv
Eval: eval1, Type: D
{'TP': 216.0, 'FN': 368.0, 'FP': 47.0, 'Precision': 0.8212927756653993, 'Recall': 0.3698630136986301, 'F1': 0.5100354191263282}
Saved to cbs081616hop128_transducer_with_lm_eval1_D.csv


100%|██████████| 8579/8579 [00:00<00:00, 168667.10it/s]


Eval: eval2, Type: F
{'TP': 2284.0, 'FN': 198.0, 'FP': 128.0, 'Precision': 0.9469320066334992, 'Recall': 0.9202256244963739, 'F1': 0.93338782182264}
Saved to cbs081616hop128_transducer_with_lm_eval2_F.csv
Eval: eval2, Type: D
{'TP': 263.0, 'FN': 290.0, 'FP': 85.0, 'Precision': 0.7557471264367817, 'Recall': 0.4755877034358047, 'F1': 0.5837957824639289}
Saved to cbs081616hop128_transducer_with_lm_eval2_D.csv


100%|██████████| 8723/8723 [00:00<00:00, 348317.42it/s]


Eval: eval3, Type: F
{'TP': 1577.0, 'FN': 123.0, 'FP': 199.0, 'Precision': 0.8879504504504504, 'Recall': 0.9276470588235294, 'F1': 0.9073647871116226}
Saved to cbs081616hop128_transducer_with_lm_eval3_F.csv
Eval: eval3, Type: D
{'TP': 93.0, 'FN': 174.0, 'FP': 31.0, 'Precision': 0.75, 'Recall': 0.34831460674157305, 'F1': 0.4757033248081842}
Saved to cbs081616hop128_transducer_with_lm_eval3_D.csv


In [9]:
import os

exp_name = "asr_train_asr_cbs_transducer_081616_hop128/cbs_20epoch_with_lm"
out_name = "cbs081616hop128_transducer_with_lm"

for eval_name in ["eval1", "eval2", "eval3"]:
    eval_dir = os.path.join("exp", exp_name, eval_name, "score_wer")
    result_file = os.path.join(eval_dir, "result.txt")
    result_dfs = make_result_df(result_file)

    for key, df in result_dfs.items():
        print(f"Eval: {eval_name}, Type: {key}")
        print(df.loc['ALL'].to_dict())

100%|██████████| 8501/8501 [00:00<00:00, 197934.81it/s]


A01M0097-A01M0097_0002761_0009322
REF: セ ー セ ー カ テ ー ノ | モ+D ド+D ル+D | モ デ ル ニ モ ト ズ ク | エ フ ゼ ロ パ タ ー ン ノ | ブ ン セ キ <SP> カ ク オ ン セ ツ ノ | ブ ン セ ツ ツ テ キ ト ク チョ ー ト
HYP: セ ー セ ー カ テ ー ノ | ***** モ デ ル モ デ ル ニ モ ト ズ ク | エ フ ゼ ロ パ タ ー ン ノ | ブ ン セ キ <SP> カ ク オ ン セ ツ ノ | ブ ン セ *** ツ テ キ ト ク チョ ー ト
A01M0097-A01M0097_0094240_0095091
REF: *** *** ネ+D コ+D シュ+D ボ+D ン+D
HYP: ナ ン カ | シ マ ス
A01M0097-A01M0097_0099760_0106406
REF: オ | エ+F ー+F | エ フ ゼ ロ パ タ ー ン オ <SP> モ チ ー テ | ア ラ ワ シ マ ス <SP> エ フ ゼ ロ パ タ ー ン ワ | ヨ コ ジ ク ニ | ジ カ ン | エ+F ー+F | タ+D テ+D
HYP: オ | エ+F ー+F | エ フ ゼ ロ パ タ ー ン オ | モ チ ー テ | ア ラ ワ シ マ ス <SP> エ フ ゼ ロ パ タ ー ン ワ | ヨ コ ジ ク ニ | ジ カ ン | エ+F ー+F | タ テ
A01M0097-A01M0097_0149408_0158109
REF: エ+F ー+F | ダ イ サ ン セ ー ワ | ハ ツ オ ン <SP> ガ <SP> グ ッ ト | サ ゲ テ | マ タ | モ ト ノ ト コ ロ エ | モ ド ッ テ ク ル <SP> カ タ チ ニ | シ マ ス ト | エ+F ー+F | イ+D | イ ス ノ | イ ミ ニ | ナ リ マ ス
HYP: エ+F ー+F | ダ イ サ ン セ ー ワ | ハ ツ オ ン <SP> ガ <SP> グ ッ ト | サ ゲ テ | マ タ | モ ト ノ ト コ ロ エ | モ ド ッ テ ク ル <SP> カ タ チ ニ | シ マ ス ト | エ+F ー+F * ****

100%|██████████| 8579/8579 [00:00<00:00, 208273.14it/s]


A01F0001-A01F0001_0076029_0081930
REF: エ+F ー+F | セ ー ブ ツ ソ ナ ー ト ユ ー ノ ワ <SP> エ+F ー+F | ショ リ ソ ク ド ヤ | オ+D | ソ ノ | オ ー キ サ ナ ド *** | イ ロ ン ナ | ダ+D イ+D テ+D <SP> イ ロ ン ナ | テ ン ニ オ イ テ | ジ ン コ ー ソ ナ ー ニ
HYP: エ+F ー+F | セ ー ブ ツ ソ ナ ー ト ユ ー ノ ワ <SP> エ+F ***** | ショ リ ソ ク ド ヤ * ***** | ソ ノ | オ ー キ サ ナ ド オ | イ ロ ン ナ | ***** ***** テ+D | イ ロ ン ナ | テ ン ニ オ イ テ | ジ ン コ ー ソ ナ ー ニ
A01F0001-A01F0001_0167649_0176220
REF: シュ ー ハ ス ー テ ー ジョ ー オ ン デ ア ル | シ ー エ フ オ ン ニ | ト モ ナ イ <SP> イ+D | ミ ジ カ イ | エ+F ー+F | シュ ー ハ ス ー ヘ ン チョ ー オ ン | エ フ エ ム オ ン オ | ト モ ナ ウ ヨ ー ナ | シ ー エ フ エ フ エ フ エ ム オ ン デ ア ル | コ ト ガ | ワ カ ッ テ イ マ ス
HYP: シュ ー ハ ス ー テ ー ジョ ー オ ン デ ア ル | シ ー エ フ オ ン ニ * ト モ ナ イ **** ***** | ミ ジ カ イ | エ+F ***** | シュ ー ハ ス ー ヘ ン チョ ー オ ン | エ フ エ ム オ ン オ | ト モ ナ ウ ヨ ー ナ | シ ー エ フ エ フ エ フ エ ム オ ン デ ア ル | コ ト ガ | ワ カ ッ テ イ マ ス
A01F0001-A01F0001_0212525_0219490
REF: マ タ | シ ー エ フ ツ ー シュ ー ハ ス ー ワ | エ コ ー ロ ケ ー ショ ン キ ノ ー オ | カ ツ ヨ ー シ テ ル カ | シ+D ネ+D ***** イ+D ネ+D | シ テ イ ナ イ カ ニ | イ ゾ ン シ <SP> ジョ ー ゲ シ テ *** 

  0%|          | 0/8723 [00:00<?, ?it/s]

S00F0019-S00F0019_0037205_0041236
REF: ト+F | マ ズ ー | ツ イ テ | オ ド ロ イ タ ノ ガ <SP> エ+F ー+F ト+F | コ ッ チ ノ | ト ー キョ ー ワ | エ+D
HYP: ト+F | マ ズ *** | ツ イ テ | オ ド ロ イ タ ノ ガ <SP> エ+F ー+F ト+F | コ ッ チ ノ | ト ー キョ ー ワ * *****
S00F0019-S00F0019_0041750_0048436
REF: マ ダ | ユ キ ワ | ゼ ン ゼ ン | フ ッ テ ナ イ ン デ ス ガ <SP> モ ー | サ ッ ポ ロ ワ | ユ キ ガ | *** ツ モ ッ テ | イ チ メ ン | マ+D ス+D シ+D | ン+F ト+F | マ ッ シ ロ デ シ タ
HYP: マ ダ | ユ キ ワ | ゼ ン ゼ ン | フ ッ テ ナ イ ン デ ス ガ <SP> モ ー | サ ッ ポ ロ ワ | ユ キ ガ | ア ツ マ ッ テ | イ チ メ ン | ***** ***** マ ス ン ト | マ ッ シ ロ デ シ タ
S00F0019-S00F0019_0082120_0088127
REF: ト+F ー+F | ハ シ *** ル ト | ウ+D <SP> モ ー | ス グ | ラ ー メ ン ヤ サ ン デ | ホ ン ト ニ | ラ ー メ ン ヤ サ ン ガ | オ ー イ ナ *** ッ テ ユ ー | カ ン ジ ガ | シ マ シ タ
HYP: ト+F ー+F | ハ シ ー ル ト | ***** **** モ ー | ス グ | ラ ー メ ン ヤ サ ン デ | ホ ン ト ニ | ラ ー メ ン ヤ サ ン ガ | オ ー イ ナ ー ッ テ ユ ー | カ ン ジ ガ | シ マ シ タ
S00F0019-S00F0019_0136514_0139912
REF: エ+F ー+F ト+F ー+F | ソ+D ー+D | ソ ノ | ア ト ニ <SP> エ+F ー+F
HYP: エ+F ー+F ト+F ー+F | ソ ー | ソ ノ | ア ト ニ <SP> エ+F ー+F
S00F0019-S00F0019_0210774_02

100%|██████████| 8723/8723 [00:00<00:00, 198591.52it/s]


Eval: eval3, Type: F
{'TP': 1577.0, 'FN': 123.0, 'FP': 199.0, 'Precision': 0.8879504504504504, 'Recall': 0.9276470588235294, 'F1': 0.9073647871116226}
Eval: eval3, Type: D
{'TP': 93.0, 'FN': 174.0, 'FP': 31.0, 'Precision': 0.75, 'Recall': 0.34831460674157305, 'F1': 0.4757033248081842}


In [2]:
import os

exp_name = "asr_train_asr_cbs_transducer_081616_hop128/cbs_final_with_lm"
out_name = "cbs081616hop128_transducer_final_with_lm"

for eval_name in ["eval1", "eval2", "eval3"]:
    eval_dir = os.path.join("exp", exp_name, eval_name, "score_wer")
    result_file = os.path.join(eval_dir, "result.txt")
    result_dfs = make_result_df(result_file)

    for key, df in result_dfs.items():
        print(f"Eval: {eval_name}, Type: {key}")
        print(df.loc['ALL'].to_dict())
        out_csv_name = f"{out_name}_{eval_name}_{key}.csv"
        df.to_csv(out_csv_name)
        print(f"Saved to {out_csv_name}")

100%|██████████| 8501/8501 [00:00<00:00, 243065.60it/s]


Eval: eval1, Type: F
{'TP': 2967.0, 'FN': 259.0, 'FP': 166.0, 'Precision': 0.947015639961698, 'Recall': 0.9197148171109734, 'F1': 0.9331655920742256}
Saved to cbs081616hop128_transducer_final_with_lm_eval1_F.csv
Eval: eval1, Type: D
{'TP': 245.0, 'FN': 339.0, 'FP': 36.0, 'Precision': 0.8718861209964412, 'Recall': 0.4195205479452055, 'F1': 0.5664739884393063}
Saved to cbs081616hop128_transducer_final_with_lm_eval1_D.csv


100%|██████████| 8576/8576 [00:00<00:00, 235078.82it/s]


Eval: eval2, Type: F
{'TP': 2298.0, 'FN': 184.0, 'FP': 123.0, 'Precision': 0.9491945477075588, 'Recall': 0.9258662369057212, 'F1': 0.9373852743218437}
Saved to cbs081616hop128_transducer_final_with_lm_eval2_F.csv
Eval: eval2, Type: D
{'TP': 289.0, 'FN': 264.0, 'FP': 63.0, 'Precision': 0.8210227272727273, 'Recall': 0.5226039783001808, 'F1': 0.6386740331491713}
Saved to cbs081616hop128_transducer_final_with_lm_eval2_D.csv


100%|██████████| 8723/8723 [00:00<00:00, 349645.58it/s]


Eval: eval3, Type: F
{'TP': 1584.0, 'FN': 116.0, 'FP': 179.0, 'Precision': 0.8984685195689166, 'Recall': 0.9317647058823529, 'F1': 0.9148137453075369}
Saved to cbs081616hop128_transducer_final_with_lm_eval3_F.csv
Eval: eval3, Type: D
{'TP': 105.0, 'FN': 162.0, 'FP': 18.0, 'Precision': 0.8536585365853658, 'Recall': 0.39325842696629215, 'F1': 0.5384615384615385}
Saved to cbs081616hop128_transducer_final_with_lm_eval3_D.csv


In [2]:
import os

exp_name = "asr_train_asr_cbs_transducer_081616_hop128/cbs_final_without_lm"
out_name = "cbs081616hop128_transducer_final_without_lm"

for eval_name in ["eval1", "eval2", "eval3"]:
    eval_dir = os.path.join("exp", exp_name, eval_name, "score_wer")
    result_file = os.path.join(eval_dir, "result.txt")
    result_dfs = make_result_df(result_file)

    for key, df in result_dfs.items():
        print(f"Eval: {eval_name}, Type: {key}")
        print(df.loc['ALL'].to_dict())
        out_csv_name = f"{out_name}_{eval_name}_{key}.csv"
        df.to_csv(out_csv_name)
        print(f"Saved to {out_csv_name}")

100%|██████████| 8501/8501 [00:00<00:00, 161857.63it/s]


Eval: eval1, Type: F
{'TP': 2975.0, 'FN': 251.0, 'FP': 176.0, 'Precision': 0.9441447159631863, 'Recall': 0.9221946683199008, 'F1': 0.9330406147091108}
Saved to cbs081616hop128_transducer_final_without_lm_eval1_F.csv
Eval: eval1, Type: D
{'TP': 313.0, 'FN': 271.0, 'FP': 85.0, 'Precision': 0.7864321608040201, 'Recall': 0.535958904109589, 'F1': 0.6374745417515274}
Saved to cbs081616hop128_transducer_final_without_lm_eval1_D.csv


100%|██████████| 8579/8579 [00:00<00:00, 231099.81it/s]


Eval: eval2, Type: F
{'TP': 2307.0, 'FN': 175.0, 'FP': 139.0, 'Precision': 0.9431725265739984, 'Recall': 0.9294923448831588, 'F1': 0.9362824675324676}
Saved to cbs081616hop128_transducer_final_without_lm_eval2_F.csv
Eval: eval2, Type: D
{'TP': 356.0, 'FN': 197.0, 'FP': 140.0, 'Precision': 0.717741935483871, 'Recall': 0.64376130198915, 'F1': 0.6787416587225928}
Saved to cbs081616hop128_transducer_final_without_lm_eval2_D.csv


100%|██████████| 8723/8723 [00:00<00:00, 314769.46it/s]


Eval: eval3, Type: F
{'TP': 1587.0, 'FN': 113.0, 'FP': 178.0, 'Precision': 0.8991501416430595, 'Recall': 0.9335294117647058, 'F1': 0.9160173160173161}
Saved to cbs081616hop128_transducer_final_without_lm_eval3_F.csv
Eval: eval3, Type: D
{'TP': 141.0, 'FN': 126.0, 'FP': 50.0, 'Precision': 0.7382198952879581, 'Recall': 0.5280898876404494, 'F1': 0.6157205240174671}
Saved to cbs081616hop128_transducer_final_without_lm_eval3_D.csv


In [ ]:
import os

exp_name = "asr_train_asr_cbs_transducer_120303_12blk/decode_cbs_transducer_lm_lm_train_lm_jp_word_valid.loss.ave_asr_model_valid.cer_transducer.ave"
out_name = "cbs120303_12blk_transducer_without_lm"

for eval_name in ["eval1", "eval2", "eval3"]:
    eval_dir = os.path.join("exp", exp_name, eval_name, "score_wer")
    result_file = os.path.join(eval_dir, "result.txt")
    result_dfs = make_result_df(result_file)

    for key, df in result_dfs.items():
        print(f"Eval: {eval_name}, Type: {key}")
        print(df.loc['ALL'].to_dict())
        out_csv_name = f"{out_name}_{eval_name}_{key}.csv"
        df.to_csv(out_csv_name)
        print(f"Saved to {out_csv_name}")